In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import *
from ipywidgets import Button, HBox, VBox

In [2]:
# Form back
name_to_code = {
    'Infosys Ltd.': 'INFY',
    'Tata Consultancy Services Ltd.': 'TCS',
    'Aptech Ltd.': 'APTECHT'
}

code_to_name = {
    'TCS': 'Tata Consultancy Services Ltd.',
    'APTECHT': 'Aptech Ltd.',
    'INFY': 'Infosys Ltd.'
}

code_to_id = {
    'INFY': 123,
    'TCS': 234,
    'APTECHT': 345
}

# labels for form entity

stock_name = widgets.Dropdown(options=['Aptech Ltd.', 'Tata Consultancy Services Ltd.',
                                       'Infosys Ltd.'])
stock_symbol = widgets.Dropdown(options=['APTECHT', 'TCS', 'INFY'])
order_type = widgets.Dropdown(options=['RL', 'SL', 'OL', 'CA'])
total_quantity = widgets.IntText()
price = widgets.FloatText()
stock_range = widgets.FloatRangeSlider(min=0, max=100000)
date = widgets.DatePicker(disabled=False)

open_tb = widgets.FloatText(value=0.0, disabled=True)
low_tb = widgets.FloatText(value=0.0, disabled=True)
high_tb = widgets.FloatText(value=0.0, disabled=True)
close_tb = widgets.FloatText(value=0.0, disabled=True)

submit_button = widgets.Button(description='Submit Order', button_style='info')


# symbol to name and vice-versa

def on_change_n2c(change):
    if change['type'] == 'change' and change['name'] == 'value':
        stock_symbol.value = name_to_code[stock_name.value]

stock_name.observe(on_change_n2c)

def on_change_c2n(change):
    if change['type'] == 'change' and change['name'] == 'value':
        stock_name.value = code_to_name[stock_symbol.value]
        
stock_symbol.observe(on_change_c2n)

# # submit function

db_dict = {}
def handle_submit(sender):
    db_dict = {
        'Stock Name': stock_name.value,
        'Stock Symbol': stock_symbol.value,
        'Stock ID': code_to_id[stock_symbol.value],
        'Order Type': order_type.value,
        'Total Quantity': total_quantity.value,
        'Price': price.value,
        'Stock Range': stock_range.value,
        'Date': date.value
    }
    
    obj = Predict(stock_name.value, stock_symbol.value, date.value)
    result = obj.predict_values()
    print(result)
    open_tb.value = result['Open']
    close_tb.value = result['Close']
    high_tb.value = result['High']
    low_tb.value = result['Low']

submit_button.on_click(handle_submit)

In [3]:
# Class
from keras.models import load_model
import keras
import pandas as pd
from dateutil import parser
from sklearn.preprocessing import MinMaxScaler

class Predict(object):
    
    lb = 7 
    
    def __init__(self, name, ticker, date):

        self.NAME    = name
        self.TICKER  = ticker
        self.DATE    = date
        self.MODEL   = self.lm()
        self.RESULTS = self.load_dictionary()
        self.DATA    = self.load_data()
        
    def load_dictionary(self):
        return {'Open' : 0,
                'Close': 0,
                'High' : 0,
                'Low'  : 0  }
    
    def load_data(self):
        table = pd.read_csv('../src/csv/'+'NSE-'+self.TICKER+'.csv')
        table['Date'] = pd.to_datetime(table['Date'])
        table = table.iloc[::-1,:]
        return table
    
    def lm(self):
        open_model  = load_model('../src/models/'+self.TICKER+'_Open.h5')
        close_model = load_model('../src/models/'+self.TICKER+'_Close.h5')
        high_model  = load_model('../src/models/'+self.TICKER+'_High.h5')
        low_model   = load_model('../src/models/'+self.TICKER+'_Low.h5')
        return {'Open' :open_model,
                'Close':close_model,
                'High' :high_model,
                'Low'  :low_model   }
    
    def predict_values(self):
        for key, value in self.RESULTS.items():
            scl = MinMaxScaler()
            data = self.DATA
            cl = self.DATA[key].values.reshape(self.DATA[key].shape[0],1)
            cl = scl.fit_transform(cl)
            indx = 0
            for _ in data.Date:
                if _.date == self.DATE:
                    indx = data.index[data.Date == _]
            test = cl[indx:indx+self.lb,0]
            test = test.reshape((1,test.shape[0],1))
            predicted_value = self.MODEL[key].predict(test)
            self.RESULTS[key] = scl.inverse_transform(predicted_value.reshape(-1,1))[0,0]
        return self.RESULTS

Using TensorFlow backend.
/Users/vatsalsmacbook/anaconda/envs/NewFrontier/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
form = VBox([
    HBox([Label('Stock Name:     '), stock_name]),
    HBox([Label('Stock Symbol:   '), stock_symbol]),
    HBox([Label('Order Type:     '), order_type]),
    HBox([Label('Total Quantity: '), total_quantity]),
    HBox([Label('Price:          '), price]),
#     HBox([Label('Client Code:    '), client_code]),
    HBox([Label('Stock Range:    '), stock_range]),
    HBox([Label('Pick a date:    '), date]),
    HBox([Label('Open: '), open_tb, Label('High: '), high_tb]),
    HBox([Label('Low: '), low_tb, Label('Close: '), close_tb]),
    HBox([submit_button]),
])

display(form)

VBox(children=(HBox(children=(Label(value='Stock Name:     '), Dropdown(options=('Aptech Ltd.', 'Tata Consultancy Services Ltd.', 'Infosys Ltd.'), value='Aptech Ltd.'))), HBox(children=(Label(value='Stock Symbol:   '), Dropdown(options=('APTECHT', 'TCS', 'INFY'), value='APTECHT'))), HBox(children=(Label(value='Order Type:     '), Dropdown(options=('RL', 'SL', 'OL', 'CA'), value='RL'))), HBox(children=(Label(value='Total Quantity: '), IntText(value=0))), HBox(children=(Label(value='Price:          '), FloatText(value=0.0))), HBox(children=(Label(value='Stock Range:    '), FloatRangeSlider(value=(25000.0, 75000.0), max=100000.0))), HBox(children=(Label(value='Pick a date:    '), DatePicker(value=None))), HBox(children=(Label(value='Open: '), FloatText(value=0.0, disabled=True), Label(value='High: '), FloatText(value=0.0, disabled=True))), HBox(children=(Label(value='Low: '), FloatText(value=0.0, disabled=True), Label(value='Close: '), FloatText(value=0.0, disabled=True))), HBox(children=(Button(button_style='info', description='Submit Order', style=ButtonStyle()),))))

{'Open': 75.520325, 'Close': 75.718185, 'High': 76.798576, 'Low': 74.917595}
{'Open': 2148.0388, 'Close': 2152.5947, 'High': 2178.8909, 'Low': 2130.9282}
